## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib.pyplot as plt

# Add Matplotlib inline magic command
%matplotlib inline

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

Deliverables:
The Vacation_Search.ipynb file
The WeatherPy_vacation.csv file
The WeatherPy_vacation_map.png image



In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users/mill2313/Weather_Database/WeatherPy_database.csv")
city_data_df.head()
# city_data_df.shape[0]

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Description
0,0,Hithadhoo,100,MV,2022-07-27 19:19:37,76,-0.6000,73.0833,81.82,10.04,overcast clouds
1,1,Busselton,7,AU,2022-07-27 19:19:37,74,-33.6500,115.3333,52.20,7.61,clear sky
2,2,Dakar,75,SN,2022-07-27 19:17:25,79,14.6937,-17.4441,84.33,6.91,broken clouds
3,3,Hobart,100,AU,2022-07-27 19:19:38,77,-42.8794,147.3294,46.26,8.05,overcast clouds
4,4,Abu Dhabi,0,AE,2022-07-27 19:19:18,69,24.4667,54.3667,89.71,5.75,clear sky


In [3]:
##### 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 67
What is the maximum temperature you would like for your trip? 76


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)
preferred_cities_df.shape[0]


158

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Cloudiness     0
Country        1
Date           0
Humidity       0
Lat            0
Lng            0
Max Temp       0
Wind Speed     0
Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
10,Vaini,TO,71.76,few clouds,-21.2000,-175.2000,
25,Lupiro,TZ,74.84,overcast clouds,-8.3833,36.6667,
32,Rikitea,PF,73.90,overcast clouds,-23.1203,-134.9692,
44,Krasnopavlivka,UA,70.12,scattered clouds,49.1372,36.3272,
48,Ambilobe,MG,68.65,light rain,-13.2000,49.0500,
50,Lagoa,PT,73.17,overcast clouds,39.0500,-27.9833,
71,Ponta Do Sol,PT,75.70,clear sky,32.6667,-17.1000,
72,Dingle,PH,75.99,overcast clouds,10.9995,122.6711,
77,Ponta Delgada,PT,75.92,few clouds,37.7333,-25.6667,
79,Sept-Iles,CA,68.00,broken clouds,50.2001,-66.3821,


In [8]:
# 6a. Set parameters to search for hotels within 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Set up the URL and use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")



# hotel_df
 

        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.

# Get names of indexes where hotel name is blank
no_hotel = hotel_df[ hotel_df['Hotel Name'] == "" ].index
no_hotel
# Delete these row indexes from dataFrame
hotel_df.drop(no_hotel , inplace=True)
clean_hotel_df = hotel_df.copy()
# clean_hotel_df.head()


In [10]:
# 8a. Create the output File (CSV)

#  Set location for the output file
clean_hotel_csv = "C:/Users/mill2313/Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(clean_hotel_csv, index_label="City_ID")

In [11]:
# # 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
# info_box_template = """

# """
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{City}</dd>
<dt>City</dt><dd>{Country}</dd>
<dt>Country</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# # 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
# locations
# hotel_info

In [13]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

#Save the map to file
plt.savefig("C:/Users/mill2313/Vacation_Search/WeatherPy_vacation_map.png")

Figure(layout=FigureLayout(height='420px'))